# 코루틴
파이썬에서 코루틴(coroutine)은 비동기 프로그래밍에 유용한 일종의 함수로, 협업 멀티태스킹을 가능하게 합니다. 코루틴은 일반 함수와 비슷하게 작동하지만, 일반 함수와는 다르게 실행을 일시 중지하고 다시 시작할 수 있습니다. 이를 통해 코루틴은 다른 코루틴이나 함수와 협력하여 여러 작업을 동시에 처리할 수 있습니다.

코루틴은 async def 키워드를 사용하여 정의됩니다. 코루틴 내에서 다른 코루틴을 호출할 때는 await 키워드를 사용합니다. 이렇게 하면 코루틴은 비동기 작업을 기다리는 동안 실행을 일시 중단하고, 이후에 다시 실행을 재개할 수 있습니다. 이러한 동작 방식 덕분에 코루틴은 네트워크 작업, 파일 I/O 등 블로킹(blocking) 작업을 수행할 때 유용하게 사용할 수 있습니다.

개요
* Coroutine 개념
* 제너레이터 vs 코루틴
* 코루틴 기초 실습
* 코루틴 예외 처리
* yield from

yield : 메인루틴과 서브 루틴 간 통신을 할 수 있게 해줌, 코루틴 제어, 코루틴 상태, 양방향 값 전송
* 메인 루틴 : 순차적으로 진행(하나의 흐름)
* 서브 루틴 : 멈춰 있어서 다른 호출된 부분으로 이동

* 서브루틴 : 메인루틴에서 -> 리턴에 의해 호출 부분으로 돌아와 다시 프로세스
* 코루틴 : 루틴 실행 중 멈춤 가능 -> 특정 위치로 돌아갔다가 -> 다시 원래 위치로 돌아와 수행 가능 -> 동시성 프로그래밍 가능 (가독성 안좋음, 여기 저기 가기에)
* 코루틴 : 코루틴 스케쥴링 오버헤드 매우 적다.
* 쓰레드 : 싱글쓰레드 -> 멀티쓰레드 -> 복잡 -> 공유되는 자원 -> 교착 상태 발생 가능성, 컨텍스트 스위칭 비용 발생, 자원 소비 가능성 증가

In [5]:
# 코루틴 예제1

def coroutine1():
    print('>>> coroutine started.')
    i = yield # main 루틴으로부터 값을 받을 수 있음
    print('>>> coroutine received : {}'.format(i))

# 제네레이터 선언

c1 = coroutine1()

print('EX1-1 -', c1, type(c1))

EX1-1 - <generator object coroutine1 at 0x7faf500696d0> <class 'generator'>


In [6]:
# yield 실행 전까지 진행
next(c1)
# 기본으로 None 전달
# next(c1)
# 값 전송
c1.send(100)

>>> coroutine started.
>>> coroutine received : 100


StopIteration: 

In [7]:
# 잘못된 사용

c2 = coroutine1()
c2.send(100) # 예외 발생

TypeError: can't send non-None value to a just-started generator

# 코루틴 예제2
* GEN_CREATED : 처음 대기 상태
* GEN_RUNNING : 실행 상태
* GEN_SUSPENDED : yield 대기 상태
* GEN_CLOSED : 실행 완료 상태

In [9]:
def coroutine2(x):
    print('>>> coroutine started : {}'.format(x))
    y = yield x
    print('>>> coroutine received : {}'.format(y))
    z = yield x + y
    print('>>> coroutine received : {}'.format(z))
    
c3 = coroutine2(10)

In [10]:
from inspect import getgeneratorstate

print('EX1-2 -', getgeneratorstate(c3))

print(next(c3))

print('EX1-3 -', getgeneratorstate(c3))

print(c3.send(15))

EX1-2 - GEN_CREATED
>>> coroutine started : 10
10
EX1-3 - GEN_SUSPENDED
>>> coroutine received : 15
25


In [11]:
# 데코레이터 패턴

from functools import wraps

def coroutine(func):
    '''Decorator run until yield'''
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def sumer():
    total = 0
    term = 0
    while True:
        term = yield total
        total += term


su = sumer()

print('EX2-1 -', su.send(100))
print('EX2-2 -', su.send(40))
print('EX2-3 -', su.send(60))

EX2-1 - 100
EX2-2 - 140
EX2-3 - 200


In [15]:
class SampleException(Exception):
    '''설명에 사용할 예외 유형'''
    
def coroutine_except():
    print('>> coroutine stated.')
    try:
        while True:
            try:
                x = yield
            except SampleException:
                print('-> SampleException handled. Continuing..')
            else:
                print('-> coroutine received : {}'.format(x))
    finally:
        print('-> coroutine ending')


exe_co = coroutine_except()

print('EX3-1 -', next(exe_co))
print('EX3-2 -', exe_co.send(10))
print('EX3-3 -', exe_co.send(100))
print('EX3-4 -', exe_co.throw(SampleException))
print('EX3-5 -', exe_co.send(1000))
print('EX3-6 -', exe_co.close()) # GEN_CLOSED

>> coroutine stated.
EX3-1 - None
-> coroutine received : 10
EX3-2 - None
-> coroutine received : 100
EX3-3 - None
-> SampleException handled. Continuing..
EX3-4 - None
-> coroutine received : 1000
EX3-5 - None
-> coroutine ending
EX3-6 - None


In [16]:
def averager_re():
    total = 0.0
    cnt = 0
    avg = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        cnt += 1
        avg = total / cnt
    return 'Average : {}'.format(avg)

avger2 =averager_re()

next(avger2)

avger2.send(10)
avger2.send(30)
avger2.send(50)


try:
    avger2.send(None)
except StopIteration as e:
    print('EX4-1 -', e.value)

EX4-1 - Average : 30.0


In [17]:
# 코루틴 예제5(yeild from)
# StopIteration 자동 처리(3.7 -> await)
# 중첩 코루틴 처리

def gen1():
    for x in 'AB':
        yield x
    for y in range(1,4):
        yield y

t1 = gen1()

print('EX5-1 -', next(t1))
print('EX5-2 -', next(t1))
print('EX5-3 -', next(t1))
print('EX5-4 -', next(t1))
print('EX5-5 -', next(t1))
# print('EX5-6 -', next(t1))

t2 = gen1()

print('EX5-7 -', list(t2))

EX5-1 - A
EX5-2 - B
EX5-3 - 1
EX5-4 - 2
EX5-5 - 3
EX5-7 - ['A', 'B', 1, 2, 3]


In [18]:
def gen2():
    yield from 'AB'
    yield from range(1,4)



t3 = gen2()

print('EX6-1 -', next(t3))
print('EX6-2 -', next(t3))
print('EX6-3 -', next(t3))
print('EX6-4 -', next(t3))
print('EX6-5 -', next(t3))

EX6-1 - A
EX6-2 - B
EX6-3 - 1
EX6-4 - 2
EX6-5 - 3


In [19]:
t4 = gen2()

print('EX6-7 -', list(t4))

EX6-7 - ['A', 'B', 1, 2, 3]


In [20]:
def gen3_sub():
    print('Sub coroutine.')
    x = yield 10
    print('Recv : ', str(x))
    x = yield 100
    print('Recv : ', str(x))

def gen4_main():
    yield from gen3_sub()

t5 = gen4_main()

print('EX7-1 -', next(t5))
print('EX7-2 -', t5.send(7))
print('EX7-2 -', t5.send(77))

Sub coroutine.
EX7-1 - 10
Recv :  7
EX7-2 - 100
Recv :  77


StopIteration: 